In [101]:
# coding: utf-8
import os
import numpy as np
import pandas as pd
from keras.layers import Dense,Input,Conv1D,MaxPooling1D,Flatten,Dropout
from keras.optimizers import Nadam
from keras.layers.advanced_activations import PReLU
from keras.callbacks import ModelCheckpoint
import datetime
from keras.models import Model
from keras import regularizers
import matplotlib.pyplot as plt
import pickle
from keras.models import model_from_json
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from matplotlib.backends.backend_pdf import PdfPages

In [102]:
# パラメータ
nb_epoch = 200                     # 学習回数
nb_classes = 1                     # 出力チャンネル数
xvalsize = 14025                   #検証データ数
valnum = 366*24+365*24
xtestsize = 365*24                   #テストデータ数
num_div = 6 
testpoint = xtestsize
trapoint = 366*24+706
valpoint_st = 366*24
valpoint_en = 706

In [103]:
length = "0.5days"

rpath = "/home/waka/resol_new/" + length

path = rpath + "/results"
os.mkdir(path)
weightpath = path + "/weight"
os.mkdir(weightpath)

# データ読み込み

In [104]:
step = 706
spot = 13


def read_data(rpath,
              xtestsize):

    print("-"*100)
    print("Reading Data")

    
    x1 = pd.read_csv(rpath + "/all_2y.csv",header = None,dtype="float16")
    X1 = x1

    x1_origin = X1
    X1 = np.array(X1)
    
    X = X1


    y = pd.read_csv("/home/waka/resol_new/suii2008_2017.csv",header = None, skiprows=1)
    y = np.array(y)
    y_origin = y
    Y = y
    Y = np.array(Y,dtype="float16")
    

    
    X_fortrain = X[:-xtestsize,:]
    Y_fortrain = Y[:-xtestsize,:]
    
    
    X_test2017 = X[-xtestsize:,:]
    Y_test2017 = Y[-xtestsize:,:]
     
    
    print("X_fortrain.shape",X_fortrain.shape)
    print("Y_fortrain.shape",Y_fortrain.shape)
    print("X_test2017.shape",X_test2017.shape)
    print("Y_test2017.shape",Y_test2017.shape)
    
    print("-"*100)
    print("Calculating Data")
    
    X_fortrain_ave = X_fortrain.mean()
    X_fortrain_scale = X_fortrain.max()-X_fortrain.min()
    
    X_fortrain = X_fortrain - X_fortrain_ave
    X_fortrain = X_fortrain / X_fortrain_scale
    
    print(X_fortrain.shape)
    
    Y_fortrain_ave = Y_fortrain.mean()
    Y_fortrain_scale = Y_fortrain.max()-Y_fortrain.min()
    
    Y_fortrain = Y_fortrain - Y_fortrain_ave
    Y_fortrain = Y_fortrain / Y_fortrain_scale
    
    return X_fortrain,Y_fortrain,X_test2017,Y_test2017,X_fortrain_ave,X_fortrain_scale,Y_fortrain_ave,Y_fortrain_scale,y_origin

In [105]:
(X_fortrain,Y_fortrain,
 X_test2017,Y_test2017,
 X_fortrain_ave,X_fortrain_scale,
 Y_fortrain_ave,Y_fortrain_scale,
 y_origin) = read_data(rpath,xtestsize)

----------------------------------------------------------------------------------------------------
Reading Data
X_fortrain.shape (78912, 1326)
Y_fortrain.shape (78912, 1)
X_test2017.shape (8760, 1326)
Y_test2017.shape (8760, 1)
----------------------------------------------------------------------------------------------------
Calculating Data
(78912, 1326)


In [106]:
# データ分割

def div_data(num_div,X_fortrain,Y_fortrain,val_label,trapoint,valpoint_st,valpoint_en):

    X_train = X_fortrain[:-trapoint,:]
    X_val = X_fortrain[-valpoint_st:-valpoint_en,:]
    Y_train = Y_fortrain[:-trapoint,:]
    Y_val = Y_fortrain[-valpoint_st:-valpoint_en,:]
    
    val_label = val_label[:-xtestsize,:]
    val_label = val_label[-valpoint_st:-valpoint_en,:]

    
    print("X_train.shape =",X_train.shape)
    print("Y_train.shape =",Y_train.shape)
    print("X_val.shape =",X_val.shape)
    print("Y_val.shape =",Y_val.shape)   
    print("val_label.shape =",val_label.shape)   
    
    
    return (X_train,Y_train,X_val,Y_val,val_label)

In [107]:
(X_train,Y_train,
 X_val,Y_val,
 val_label) = div_data(num_div,X_fortrain,Y_fortrain,y_origin,trapoint,valpoint_st,valpoint_en)   

length,width = X_train.shape

X_train.shape = (69422, 1326)
Y_train.shape = (69422, 1)
X_val.shape = (8078, 1326)
Y_val.shape = (8078, 1)
val_label.shape = (8078, 1)


In [108]:
# 学習

def train(length,width,path,weightpath,X_train,Y_train,X_val,Y_val,nb_epoch):
    
    # 各層のパラメータ
    allcon_nodes1 = 1024                             #　全結合層ノード数                     　　
    allcon_nodes2 = 1024                              #　全結合層ノード数                     　　
    allcon_nodes3 = 64                             #　全結合層ノード数      
    filters1 = 32
    filters2 = 64
    filters3 = 128
    kernel_size = 24
    pool_size = 2
    
    batch_size = 100                                                # バッチサイズ
    optimizer = Nadam(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.4)

    print("-"*100)
    print("Building Model")

    
    inputs = Input(shape=(width,))
    print(inputs)
    
    x = Dense(allcon_nodes1, kernel_regularizer = regularizers.l1(1e-8))(inputs)
    x = (PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=None))(x)
    #x = Dropout(0.1)(x)
    x = Dense(allcon_nodes1)(x)
    x = (PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=None))(x)
    #x = Dropout(0.5)(x)
    x = Dense(allcon_nodes1)(x)
    x = (PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=None))(x)    #x = (PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=None))(x)
    #x = Dropout(0.25)(x)
    predictions = Dense(1)(x)
    
    #os.chdir(weightpath)
    fpath = '/weights.{epoch:05d}-{val_loss:.10f}.h5'
    cp_cb = ModelCheckpoint(filepath = weightpath + fpath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=True, mode='auto')
    
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    time_st = datetime.datetime.today()
    print(time_st)
    history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, validation_data=(X_val, Y_val), callbacks=[cp_cb])
    time_en = datetime.datetime.today()
    print(time_en)
    
    return history,time_st,time_en,model

In [ ]:
print("-"*100)


history,time_st,time_en,model = train(length,width,path,weightpath,X_train,Y_train,X_val,Y_val,nb_epoch)

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Building Model
Tensor("input_6:0", shape=(?, 1326), dtype=float32)
2019-03-07 11:49:45.860178
Train on 69422 samples, validate on 8078 samples
Epoch 1/200
69422/69422 [==============================] - 7s 94us/step - loss: 4.6016e-04 - val_loss: 5.7242e-04

Epoch 00001: saving model to /home/waka/resol_new/0.5days/results/weight/weights.00001-0.0005724167.h5
Epoch 2/200
69422/69422 [==============================] - 5s 77us/step - loss: 3.1845e-04 - val_loss: 4.5773e-04

Epoch 00002: saving model to /home/waka/resol_new/0.5days/results/weight/weights.00002-0.0004577265.h5
Epoch 3/200
69422/69422 [==============================] - 5s 77us/step - loss: 2.5205e-04 - val_loss: 3.9017e-04

Epoch 00003: saving model to /home/waka/resol_new/0.5days/results/weight/weights.00003-0.0003901738.h5
E

In [ ]:
# 学習結果保存

def plot_save(path,weightpath,history,model,time_st,time_en):
    #os.chdir(path)

    print("-"*100)
    print("Saving Results")

    pdf = PdfPages(path + "/loss.pdf")

    plt.figure()
    loss     = history.history['loss']
    val_loss = history.history['val_loss']
    nb_epoch = len(loss)
    plt.plot(range(nb_epoch), loss, label='loss')
    plt.plot(range(nb_epoch), val_loss, label='val_loss')
    #plt.xlim([0,200])
    #plt.ylim([0,0.0006])
    plt.grid()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.tick_params(labelsize=8)
    plt.tight_layout()
    plt.gca().ticklabel_format(style="sci", scilimits=(0,0), axis="y")
    pdf.savefig()
    
    plt.figure()
    loss     = history.history['loss']
    val_loss = history.history['val_loss']
    nb_epoch = len(loss)
    plt.plot(range(nb_epoch), loss, label='loss')
    plt.plot(range(nb_epoch), val_loss, label='val_loss')
    #plt.xlim([0,200])
    plt.ylim([0,3e-4])
    plt.grid()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.tick_params(labelsize=8)
    plt.tight_layout()
    plt.gca().ticklabel_format(style="sci", scilimits=(0,0), axis="y")
    pdf.savefig()

###
    
    plt.figure()
    loss     = history.history['loss']
    val_loss = history.history['val_loss']
    nb_epoch = len(loss)
    plt.plot(range(nb_epoch), loss, label='loss')
    plt.plot(range(nb_epoch), val_loss, label='val_loss')
    plt.xlim([0,200])
    #plt.ylim([0,0.0006])
    plt.grid()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.tick_params(labelsize=8)
    plt.tight_layout()
    plt.gca().ticklabel_format(style="sci", scilimits=(0,0), axis="y")
    pdf.savefig()
    
    plt.figure()
    loss     = history.history['loss']
    val_loss = history.history['val_loss']
    nb_epoch = len(loss)
    plt.plot(range(nb_epoch), loss, label='loss')
    plt.plot(range(nb_epoch), val_loss, label='val_loss')
    plt.xlim([0,200])
    plt.ylim([0,3e-4])
    plt.grid()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.tick_params(labelsize=8)
    plt.tight_layout()
    plt.gca().ticklabel_format(style="sci", scilimits=(0,0), axis="y")
    pdf.savefig()

    
    pdf.close()
    
    
    ##################
    model_json_str = model.to_json()
    open(path + "/model.json", "w").write(model_json_str)
    model.save_weights(path + "/weight_last.h5");
    ##################
    with open(path + "/history.pkl", mode = "wb") as f:
        pickle.dump(history.history, f)   
    
    with open(path + "/history.pkl", mode = "rb") as f:
        a = pickle.load(f)
        
    df = pd.DataFrame(a)
    df.to_csv(path + "/history.csv")
    
    from contextlib import redirect_stdout 
    
    with open(path + '/modelsummary.txt', 'w') as f:
        with redirect_stdout(f):
            model.summary()
            
    
    ###############################################################
    ###############################################################
    ###############################################################
    f = open(path + '/time.csv', 'w')
    f.write("time_st = " + str(time_st))
    f.write("\ntime_en = " + str(time_en))
    f.close()
    
    
    print("Finished!!!!!!!!!!!!!!!")
    print("Good Job!!!!!!!!!!!!!!!")
    
    return 

In [ ]:
print("-"*100)


plot_save(path,weightpath,history,model,time_st,time_en)

In [ ]:
###############################################################
###############################################################
###############################################################
##predict##

# predict保存_csv


def shimanto_prediction(st,
                        en,
                        path,
                        weightpath,
                        X_val,
                        Y_val,
                        X_test2017,
                        Y_test2017,
                        X_fortrain_ave,
                        X_fortrain_scale,
                        Y_fortrain_ave,
                        Y_fortrain_scale,
                        val_label):

    print("-"*100)
    print("Predicing Validation and Test")

    
    Y_val_org = val_label
    
    #X_val = X_val - X_fortrain_ave
    #X_val = X_val / X_fortrain_scale
    
    #Y_val = Y_val - Y_fortrain_ave
    #Y_val = Y_val / Y_fortrain_scale
    
    
    
    Y_test2017_org = Y_test2017
    
    X_test2017 = X_test2017 - X_fortrain_ave
    X_test2017 = X_test2017 / X_fortrain_scale
    
    Y_test2017 = Y_test2017 - Y_fortrain_ave
    Y_test2017 = Y_test2017 / Y_fortrain_scale

    w=[i for i in range(st,en)]
    # predict

    path_val = path + "/val"
    os.mkdir(path_val)
    
    
    #_val_test#
    #os.chdir(path)
    model = model_from_json(open(path + "/model.json").read())
    wlist = os.listdir(weightpath)
    wlist.sort()
    pred_all = np.zeros((len(X_val),en-st))
    c = -1
    for num in w:
        model.load_weights(weightpath + "/" + wlist[num])
        y_pred = model.predict(X_val)
    
#        te = pd.DataFrame(Y_val)
        pr = pd.DataFrame(y_pred)
    
#        label = te[0]*Y_fortrain_scale + Y_fortrain_ave
        predict = pr[0]*Y_fortrain_scale + Y_fortrain_ave
    
#        label = np.array(label)
        predict = np.array(predict)
        
        c += 1
        pred_all[:,c] = predict
        
    # 全pred    
    label = np.array(Y_val_org)
    label = label.reshape(len(label),1)
    # 平均pred    
    ave_all = np.zeros((len(label),1))
    for i in range(len(label)):
        ave_all[i,0] = pred_all[i,:].astype(float).mean()
    
    label_pred = np.hstack((label,ave_all,pred_all))
    label_pred = label_pred.astype(float)
    head = ["label","ave"]+w
    label_pred = np.vstack((head,label_pred))
    label_pred_val = pd.DataFrame(label_pred)    
    label_pred_val.to_csv(path_val + "/label_pred_all_val.csv", header = None,index = None)



    
    path2017 = path + "/2017"
    os.mkdir(path2017)
    
    
    #2017_test#
    #os.chdir(path)
    model = model_from_json(open(path + "/model.json").read())
    wlist = os.listdir(weightpath)
    wlist.sort()
    pred_all = np.zeros((len(X_test2017),en-st))
    c = -1
    for num in w:
        model.load_weights(weightpath + "/" + wlist[num])
        y_pred = model.predict(X_test2017)
    
#        te = pd.DataFrame(Y_test2017)
        pr = pd.DataFrame(y_pred)
    
#        label = te[0]*Y_fortrain_scale + Y_fortrain_ave
        predict = pr[0]*Y_fortrain_scale + Y_fortrain_ave
    
#        label = np.array(label)
        predict = np.array(predict)
        
        c += 1
        pred_all[:,c] = predict
        
    # 全pred    
    label = np.array(Y_test2017_org)
    label = label.reshape(len(label),1)
    # 平均pred    
    ave_all = np.zeros((len(label),1))
    for i in range(len(label)):
        ave_all[i,0] = pred_all[i,:].astype(float).mean()
    
    label_pred = np.hstack((label,ave_all,pred_all))
    label_pred = label_pred.astype(float)
    head = ["label","ave"]+w
    label_pred = np.vstack((head,label_pred))
    label_pred2017 = pd.DataFrame(label_pred)    
    label_pred2017.to_csv(path2017 + "/label_pred_all_2017.csv", header = None,index = None)
    
    
        
    return path_val,path2017,label_pred_val,label_pred2017

In [ ]:
# 何番目の重みを使用するか
st = 150    # 最初
en = 200    # 最後
path_val,path2017,label_pred_val,label_pred2017 = \
shimanto_prediction(st,en,path,weightpath,X_val,Y_val,X_test2017,Y_test2017,X_fortrain_ave,X_fortrain_scale,Y_fortrain_ave,Y_fortrain_scale,val_label)

In [ ]:
                    
# predict_プロット
#path_val="/home/owner/waka/shimanto/program_results/val_2016/pred_0h/L1_1e-8/val"
#path2017="/home/owner/waka/shimanto/program_results/val_2016/pred_0h/L1_1e-8/2017"
#label_pred_val=pd.read_csv(path_val+"/label_pred_all_val.csv",header=None)
#label_pred2017=pd.read_csv(path2017+"/label_pred_all_2017.csv",header=None)


def save_shimanto_results(st,en,path_val,path2017,label_pred_val,label_pred2017):
    
    print("-"*100)
    print("Ploting Outputs")

    data = pd.read_csv("/home/waka/resol_new/ave_2016_level_rain.csv",header = None,skiprows = 0)
    rai = np.array(data.iloc[:,1]).reshape(len(data),1)
    ##_val##
        
    label_pred_val = np.array(label_pred_val.iloc[1:,:]).astype(float)

    label = label_pred_val[:,0]
    pred_ave = label_pred_val[:,1]
    pred_all = label_pred_val[:,2:]
    
    
    pdf = PdfPages(path_val + "/plot_per_15days.pdf")
    
    plt.figure()    
    plt.plot(label,label="label",c="black")
    plt.plot(pred_ave,label="pred",c="red")
    plt.title("all", fontsize = 10)
    plt.legend()
    pdf.savefig()
    
    
        
    for i in range(22):
        t = range(i*24*15,(i+1)*24*15)
        plt.figure()
        plt.plot(label_pred_val[t,0],c="black",linewidth = 1.5)
        plt.plot(label_pred_val[t,2:])
        plt.ylim([-70,label.max()+100])
        plt.title(str(i+1))
        pdf.savefig()
        
    for i in range(22):
        t = range(i*24*15,(i+1)*24*15)
        plt.figure()
        plt.plot(label_pred_val[t,0],label="label",c="black",linewidth = 1.5)
        plt.plot(label_pred_val[t,1],label="pred",c="red")
        plt.ylim([-70,label.max()+100])
        plt.title(str(i+1))
        plt.legend()
        pdf.savefig()

    plt.figure(figsize=(7,7))
    x = np.arange(label_pred_val[:,0].min(),label_pred_val[:,0].max(),0.001)
    y = x
    plt.xlim([-70,label_pred_val[:,0].max()+100])
    plt.ylim([-70,label_pred_val[:,0].max()+100])
    plt.plot(x,y,c="red")
    plt.scatter(label_pred_val[:,0],label_pred_val[:,1],s=10,c="black")
    plt.xlabel("observation(cm)")
    plt.ylabel("prediction(cm)")
    plt.title("RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred_val[:,0],label_pred_val[:,1]))), fontsize = 10)
    plt.xticks(np.arange(-100,label_pred_val[:,0].max()+100, 100))
    plt.yticks(np.arange(-100,label_pred_val[:,0].max()+100, 100))
    plt.grid()
    plt.tight_layout()
    plt.tick_params(labelsize=10)
    pdf.savefig()
    
    plt.figure(figsize=(7,7))
    x = np.arange(label_pred_val[:,0].min(),label_pred_val[:,0].max(),0.001)
    y = x
    plt.xlim([-70,label_pred_val[:,0].max()+100])
    plt.ylim([-70,label_pred_val[:,0].max()+100])
    plt.plot(x,y,c="red")
    #plt.scatter(label_pred_val[:,0],label_pred_val[:,1],s=5)
    plt.plot(label_pred_val[:,0],label_pred_val[:,1],c="black",linewidth = 1.5)
    plt.xlabel("observation(cm)")
    plt.ylabel("prediction(cm)")
    plt.title("RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred_val[:,0],label_pred_val[:,1]))), fontsize = 10)
    plt.xticks(np.arange(-100,label_pred_val[:,0].max()+100, 100))
    plt.yticks(np.arange(-100,label_pred_val[:,0].max()+100, 100))
    plt.grid()
    plt.tight_layout()
    plt.tick_params(labelsize=10)
    pdf.savefig()
    
    
    plt.figure()    
    maxi = label_pred_val[:,0].max()
    maxw = np.where(label_pred_val[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred_val[:,1].max()
    maxw_p = np.where(label_pred_val[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    ad_st = maxw-24
    ad_en = maxw+24+1
    plt.figure()    
    plt.plot(label_pred_val[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    plt.plot(label_pred_val[ad_st:ad_en,1],label="pred",c="red",linewidth = 1.5)
    plt.xlim([0,len(label_pred_val[ad_st:ad_en,0])])
    plt.ylim([-70,label_pred_val[:,0].max()+100])
    plt.title("peak±24h")    
    plt.text(3,900,"value_diff=" + "{0:.3f}".format(maxi-maxi_p) + "(cm)")
    plt.text(3,800,"time_diff=" + str(maxw-maxw_p))   
    plt.text(3,1000,"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred_val[ad_st:ad_en,0],label_pred_val[ad_st:ad_en,1]))))
    plt.legend()
    pdf.savefig()

###

    #2軸#
    
    
    
    fig, ax1 = plt.subplots()
    maxi = label_pred_val[:,0].max()
    maxw = np.where(label_pred_val[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred_val[:,1].max()
    maxw_p = np.where(label_pred_val[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    ad_st = maxw-50
    ad_en = maxw+50+1
    ax1.plot(label_pred_val[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    ax1.plot(label_pred_val[ad_st:ad_en,1],label="pred",c="red",linewidth = 1.5)
    ax1.set_yticks([0,200,400,600,800,1000])
    ax1.set_ylim([-70,label_pred_val[:,0].max()+100])
    ax2 = ax1.twinx()  # 2つのプロットを関連付ける
    nnn=[i for i in range(len(rai[ad_st:ad_en,0]))]
    ax2.bar(nnn,rai[ad_st:ad_en,0],align="center",label="ave")
    ax2.set_yticks([0,20,40,60,80])
    ax2.set_ylim([80,0])
    plt.xlim([0,len(label_pred_val[ad_st:ad_en,0])])
    plt.title("peak±50h"+"  "+"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred_val[ad_st:ad_en,0],label_pred_val[ad_st:ad_en,1]))))    
    #ax2.legend()
    pdf.savefig()

###

    
    
    plt.figure()    
    maxi = label_pred_val[:,0].max()
    maxw = np.where(label_pred_val[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred_val[:,1].max()
    maxw_p = np.where(label_pred_val[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    ad_st = maxw-24
    ad_en = maxw+24+1
    plt.figure()    
    plt.plot(label_pred_val[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    plt.plot(label_pred_val[ad_st:ad_en,1],label="pred",c="red",marker=".",linestyle="None")
    plt.xlim([0,len(label_pred_val[ad_st:ad_en,0])])
    plt.ylim([-70,label_pred_val[:,0].max()+100])
    plt.title("peak±24h")    
    plt.text(3,900,"value_diff=" + "{0:.3f}".format(maxi-maxi_p) + "(cm)")
    plt.text(3,800,"time_diff=" + str(maxw-maxw_p))   
    plt.text(3,1000,"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred_val[ad_st:ad_en,0],label_pred_val[ad_st:ad_en,1]))))
    plt.legend()
    pdf.savefig()
    
    ##
    #2軸#
    
    
    
    fig, ax1 = plt.subplots()
    maxi = label_pred_val[:,0].max()
    maxw = np.where(label_pred_val[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred_val[:,1].max()
    maxw_p = np.where(label_pred_val[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    ad_st = maxw-24
    ad_en = maxw+24+1
    ax1.plot(label_pred_val[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    ax1.plot(label_pred_val[ad_st:ad_en,1],label="pred",c="red",marker=".",linestyle="None")
    ax1.set_yticks([0,200,400,600,800,1000])
    ax1.set_ylim([-70,label_pred_val[:,0].max()+100])
    ax2 = ax1.twinx()  # 2つのプロットを関連付ける
    nnn=[i for i in range(len(rai[ad_st:ad_en,0]))]
    ax2.bar(nnn,rai[ad_st:ad_en,0],align="center",label="ave")
    ax2.set_yticks([0,20,40,60,80])
    ax2.set_ylim([80,0])
    plt.xlim([0,len(label_pred_val[ad_st:ad_en,0])])
    plt.title("peak±50h"+"  "+"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred_val[ad_st:ad_en,0],label_pred_val[ad_st:ad_en,1]))))    
    #ax2.legend()
    pdf.savefig()
    
    ###
    
    
    
    
    plt.figure()    
    maxi = label_pred_val[:,0].max()
    maxw = np.where(label_pred_val[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred_val[:,1].max()
    maxw_p = np.where(label_pred_val[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    ad_st = maxw-50
    ad_en = maxw+50+1
    plt.figure()    
    plt.plot(label_pred_val[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    plt.plot(label_pred_val[ad_st:ad_en,1],label="pred",c="red",marker=".",linestyle="None")
    plt.xlim([0,len(label_pred_val[ad_st:ad_en,0])])
    plt.title("peak±50h")    
    plt.ylim([-70,label_pred_val[:,0].max()+100])
    plt.title("peak±24h")    
    plt.text(5,900,"value_diff=" + "{0:.3f}".format(maxi-maxi_p))
    plt.text(5,800,"time_diff=" + str(maxw-maxw_p))   
    plt.text(5,1000,"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred_val[ad_st:ad_en,0],label_pred_val[ad_st:ad_en,1]))))
    plt.legend()
    pdf.savefig()
    
    ###
    #2軸#
    
    
    
    fig, ax1 = plt.subplots()
    maxi = label_pred_val[:,0].max()
    maxw = np.where(label_pred_val[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred_val[:,1].max()
    maxw_p = np.where(label_pred_val[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    ad_st = maxw-50
    ad_en = maxw+50+1
    ax1.plot(label_pred_val[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    ax1.plot(label_pred_val[ad_st:ad_en,1],label="pred",c="red",marker=".",linestyle="None")
    ax1.set_yticks([0,200,400,600,800,1000])
    ax1.set_ylim([-70,label_pred_val[:,0].max()+100])
    ax2 = ax1.twinx()  # 2つのプロットを関連付ける
    nnn=[i for i in range(len(rai[ad_st:ad_en,0]))]
    ax2.bar(nnn,rai[ad_st:ad_en,0],align="center",label="ave")
    ax2.set_yticks([0,20,40,60,80])
    ax2.set_ylim([80,0])
    plt.xlim([0,len(label_pred_val[ad_st:ad_en,0])])
    plt.title("peak±50h"+"  "+"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred_val[ad_st:ad_en,0],label_pred_val[ad_st:ad_en,1]))))    
    #ax2.legend()
    pdf.savefig()

    
    pdf.close()
    
    
    ##2017##
    
    
    label_pred2017 = np.array(label_pred2017.iloc[1:,:]).astype(float)
    
    label = label_pred2017[:,0]
    pred_ave = label_pred2017[:,1]
    pred_all = label_pred2017[:,2:]


    data = pd.read_csv("/home/waka/resol_new/ave_2017_level_rain.csv",header = None,skiprows = 0)
    lev = np.array(data.iloc[:,1])
    rai = np.array(data.iloc[:,2]).reshape(len(data),1)


    pdf = PdfPages(path2017 + "/plot_per_15days.pdf")
    
    plt.figure()    
    
    plt.plot(label,label="label",c="black")
    plt.plot(pred_ave,label="pred",c="red")
    plt.title("all", fontsize = 10)
    plt.legend()
    pdf.savefig()
    
    
        
    for i in range(24):
        t = range(i*24*15,(i+1)*24*15)
        plt.figure()
        plt.plot(label_pred2017[t,0],c="black",linewidth = 1.5)
        plt.plot(label_pred2017[t,1:])
        plt.ylim([-70,label_pred2017[:,0].max()+100])
        plt.title(str(i+1))
        pdf.savefig()
        
    for i in range(24):
        t = range(i*24*15,(i+1)*24*15)
        plt.figure()
        plt.plot(label_pred2017[t,0],label="label",c="black",linewidth = 1.5)
        plt.plot(label_pred2017[t,1],label="pred",c="red",linewidth = 1.5)
        plt.ylim([-70,label_pred2017[:,0].max()+100])
        plt.title(str(i+1))
        plt.legend()
        pdf.savefig()

    
    plt.figure(figsize=(7,7))
    x = np.arange(label_pred2017[:,0].min(),label_pred2017[:,0].max(),0.001)
    y = x
    plt.xlim([-70,label_pred2017[:,0].max()+100])
    plt.ylim([-70,label_pred2017[:,0].max()+100])
    plt.plot(x,y,c="red")
    plt.scatter(label_pred2017[:,0],label_pred2017[:,1],s=10,c="black")
    plt.xlabel("observation(cm)")
    plt.ylabel("prediction(cm)")
    plt.title("RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred2017[:,0],label_pred2017[:,1]))), fontsize = 10)
    plt.xticks(np.arange(-100,label_pred2017[:,0].max()+100, 100))
    plt.yticks(np.arange(-100,label_pred2017[:,0].max()+100, 100))
    plt.grid()
    plt.tight_layout()
    plt.tick_params(labelsize=10)
    pdf.savefig()
    
    plt.figure(figsize=(7,7))
    x = np.arange(label_pred2017[:,0].min(),label_pred2017[:,0].max(),0.001)
    y = x
    plt.xlim([-70,label_pred2017[:,0].max()+100])
    plt.ylim([-70,label_pred2017[:,0].max()+100])
    plt.plot(x,y,c="red")
    #plt.scatter(label_pred2017[:,0],label_pred2017[:,1],s=5)
    plt.plot(label_pred2017[:,0],label_pred2017[:,1],c="black",linewidth = 1.5)
    plt.xlabel("observation(cm)")
    plt.ylabel("prediction(cm)")
    plt.title("RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred2017[:,0],label_pred2017[:,1]))), fontsize = 10)
    plt.xticks(np.arange(-100,label_pred2017[:,0].max()+100, 100))
    plt.yticks(np.arange(-100,label_pred2017[:,0].max()+100, 100))
    plt.grid()
    plt.tight_layout()
    plt.tick_params(labelsize=10)
    pdf.savefig()
    
    plt.figure()    
    maxi = 565
    maxw = np.where(label_pred2017[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred2017[5200:5300,1].max()
    maxw_p = np.where(label_pred2017[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    print(maxi_p)
    print(maxw_p)
    ad_st = maxw-24
    ad_en = maxw+24+1
    plt.figure()    
    plt.plot(label_pred2017[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    plt.plot(label_pred2017[ad_st:ad_en,1],label="pred",c="red",linewidth = 1.5)
    plt.xlim([0,len(label_pred2017[ad_st:ad_en,0])])
    plt.ylim([-70,label_pred2017[:,0].max()+100])
    plt.title("peak±24h"+"  "+"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred2017[ad_st:ad_en,0],label_pred2017[ad_st:ad_en,1]))))
    plt.legend()
    pdf.savefig()
    
    plt.figure()    
    maxi = 753
    maxw = np.where(label_pred2017[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred2017[5200:5300,1].max()
    maxw_p = np.where(label_pred2017[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    print(maxi_p)
    print(maxw_p)
    ad_st = maxw-24
    ad_en = maxw+24+1
    plt.figure()    
    plt.plot(label_pred2017[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    plt.plot(label_pred2017[ad_st:ad_en,1],label="pred",c="red",linewidth = 1.5)
    plt.xlim([0,len(label_pred2017[ad_st:ad_en,0])])
    plt.ylim([-70,label_pred2017[:,0].max()+100])
    plt.title("peak±24h"+"  "+"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred2017[ad_st:ad_en,0],label_pred2017[ad_st:ad_en,1]))))    
    plt.legend()
    pdf.savefig()
    
    plt.figure()    
    maxi = label_pred2017[:,0].max()
    maxw = np.where(label_pred2017[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred2017[:,1].max()
    maxw_p = np.where(label_pred2017[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    ad_st = maxw-24
    ad_en = maxw+24+1
    plt.figure()    
    plt.plot(label_pred2017[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    plt.plot(label_pred2017[ad_st:ad_en,1],label="pred",c="red",linewidth = 1.5)
    plt.xlim([0,len(label_pred2017[ad_st:ad_en,0])])
    plt.ylim([-70,label_pred2017[:,0].max()+100])
    plt.title("peak±24h")    
    plt.text(3,900,"value_diff=" + "{0:.3f}".format(maxi-maxi_p) + "(cm)")
    plt.text(3,800,"time_diff=" + str(maxw-maxw_p))   
    plt.text(3,1000,"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred2017[ad_st:ad_en,0],label_pred2017[ad_st:ad_en,1]))))
    plt.legend()
    pdf.savefig()

###

    #2軸#
    
    
    fig, ax1 = plt.subplots()
    maxi = 565
    maxw = np.where(label_pred2017[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred2017[5200:5300,1].max()
    maxw_p = np.where(label_pred2017[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    print(maxi_p)
    print(maxw_p)
    ad_st = maxw-50
    ad_en = maxw+50+1
    ax1.plot(label_pred2017[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    ax1.plot(label_pred2017[ad_st:ad_en,1],label="pred",c="red",linewidth = 1.5)
    ax1.set_yticks([0,200,400,600,800,1000])
    ax1.set_ylim([-70,label_pred2017[:,0].max()+100])
    ax2 = ax1.twinx()  # 2つのプロットを関連付ける
    nnn=[i for i in range(len(rai[ad_st:ad_en,0]))]
    ax2.bar(nnn,rai[ad_st:ad_en,0],align="center",label="ave")
    ax2.set_yticks([0,20,40,60,80])
    ax2.set_ylim([80,0])
    plt.xlim([0,len(label_pred2017[ad_st:ad_en,0])])
    plt.title("peak±50h")    
    #ax2.legend()
    pdf.savefig()
    
    
    fig, ax1 = plt.subplots()
    maxi = 753
    maxw = np.where(label_pred2017[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred2017[5200:5300,1].max()
    maxw_p = np.where(label_pred2017[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    print(maxi_p)
    print(maxw_p)
    ad_st = maxw-50
    ad_en = maxw+50+1
    ax1.plot(label_pred2017[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    ax1.plot(label_pred2017[ad_st:ad_en,1],label="pred",c="red",linewidth = 1.5)
    ax1.set_yticks([0,200,400,600,800,1000])
    ax1.set_ylim([-70,label_pred2017[:,0].max()+100])
    ax2 = ax1.twinx()  # 2つのプロットを関連付ける
    nnn=[i for i in range(len(rai[ad_st:ad_en,0]))]
    ax2.bar(nnn,rai[ad_st:ad_en,0],align="center",label="ave")
    ax2.set_yticks([0,20,40,60,80])
    ax2.set_ylim([80,0])
    plt.xlim([0,len(label_pred2017[ad_st:ad_en,0])])
    plt.title("peak±50h")    
    #ax2.legend()
    pdf.savefig()
    
    
    fig, ax1 = plt.subplots()
    maxi = label_pred2017[:,0].max()
    maxw = np.where(label_pred2017[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred2017[:,1].max()
    maxw_p = np.where(label_pred2017[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    ad_st = maxw-50
    ad_en = maxw+50+1
    ax1.plot(label_pred2017[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    ax1.plot(label_pred2017[ad_st:ad_en,1],label="pred",c="red",linewidth = 1.5)
    ax1.set_yticks([0,200,400,600,800,1000])
    ax1.set_ylim([-70,label_pred2017[:,0].max()+100])
    ax2 = ax1.twinx()  # 2つのプロットを関連付ける
    nnn=[i for i in range(len(rai[ad_st:ad_en,0]))]
    ax2.bar(nnn,rai[ad_st:ad_en,0],align="center",label="ave")
    ax2.set_yticks([0,20,40,60,80])
    ax2.set_ylim([80,0])
    plt.xlim([0,len(label_pred2017[ad_st:ad_en,0])])
    plt.title("peak±50h")    
    #ax2.legend()
    pdf.savefig()

###

    
    plt.figure()    
    maxi = 565
    maxw = np.where(label_pred2017[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred2017[5200:5300,1].max()
    maxw_p = np.where(label_pred2017[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    print(maxi_p)
    print(maxw_p)
    ad_st = maxw-24
    ad_en = maxw+24+1
    plt.figure()    
    plt.plot(label_pred2017[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    plt.plot(label_pred2017[ad_st:ad_en,1],label="pred",c="red",marker=".",linestyle="None")
    plt.xlim([0,len(label_pred2017[ad_st:ad_en,0])])
    plt.ylim([-70,label_pred2017[:,0].max()+100])
    plt.title("peak±24h"+"  "+"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred2017[ad_st:ad_en,0],label_pred2017[ad_st:ad_en,1]))))    
    plt.legend()
    pdf.savefig()
    
    plt.figure()    
    maxi = 753
    maxw = np.where(label_pred2017[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred2017[5200:5300,1].max()
    maxw_p = np.where(label_pred2017[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    print(maxi_p)
    print(maxw_p)
    ad_st = maxw-24
    ad_en = maxw+24+1
    plt.figure()    
    plt.plot(label_pred2017[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    plt.plot(label_pred2017[ad_st:ad_en,1],label="pred",c="red",marker=".",linestyle="None")
    plt.xlim([0,len(label_pred2017[ad_st:ad_en,0])])
    plt.ylim([-70,label_pred2017[:,0].max()+100])
    plt.title("peak±24h"+"  "+"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred2017[ad_st:ad_en,0],label_pred2017[ad_st:ad_en,1]))))    
    plt.legend()
    pdf.savefig()
    
    plt.figure()    
    maxi = label_pred2017[:,0].max()
    maxw = np.where(label_pred2017[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred2017[:,1].max()
    maxw_p = np.where(label_pred2017[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    ad_st = maxw-24
    ad_en = maxw+24+1
    plt.figure()    
    plt.plot(label_pred2017[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    plt.plot(label_pred2017[ad_st:ad_en,1],label="pred",c="red",marker=".",linestyle="None")
    plt.xlim([0,len(label_pred2017[ad_st:ad_en,0])])
    plt.ylim([-70,label_pred2017[:,0].max()+100])
    plt.title("peak±24h")    
    plt.text(3,900,"value_diff=" + "{0:.3f}".format(maxi-maxi_p) + "(cm)")
    plt.text(3,800,"time_diff=" + str(maxw-maxw_p))   
    plt.text(3,1000,"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred2017[ad_st:ad_en,0],label_pred2017[ad_st:ad_en,1]))))
    plt.legend()
    pdf.savefig()
    
    ##
    #2軸#
    
    
    fig, ax1 = plt.subplots()
    maxi = 565
    maxw = np.where(label_pred2017[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred2017[5200:5300,1].max()
    maxw_p = np.where(label_pred2017[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    print(maxi_p)
    print(maxw_p)
    ad_st = maxw-24
    ad_en = maxw+24+1
    ax1.plot(label_pred2017[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    ax1.plot(label_pred2017[ad_st:ad_en,1],label="pred",c="red",marker=".",linestyle="None")
    ax1.set_yticks([0,200,400,600,800,1000])
    ax1.set_ylim([-70,label_pred2017[:,0].max()+100])
    ax2 = ax1.twinx()  # 2つのプロットを関連付ける
    nnn=[i for i in range(len(rai[ad_st:ad_en,0]))]
    ax2.bar(nnn,rai[ad_st:ad_en,0],align="center",label="ave")
    ax2.set_yticks([0,20,40,60,80])
    ax2.set_ylim([80,0])
    plt.xlim([0,len(label_pred2017[ad_st:ad_en,0])])
    plt.title("peak±24h"+"  "+"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred2017[ad_st:ad_en,0],label_pred2017[ad_st:ad_en,1]))))    
    #ax2.legend()
    pdf.savefig()
    
    
    fig, ax1 = plt.subplots()
    maxi = 753
    maxw = np.where(label_pred2017[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred2017[5200:5300,1].max()
    maxw_p = np.where(label_pred2017[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    print(maxi_p)
    print(maxw_p)
    ad_st = maxw-24
    ad_en = maxw+24+1
    ax1.plot(label_pred2017[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    ax1.plot(label_pred2017[ad_st:ad_en,1],label="pred",c="red",marker=".",linestyle="None")
    ax1.set_yticks([0,200,400,600,800,1000])
    ax1.set_ylim([-70,label_pred2017[:,0].max()+100])
    ax2 = ax1.twinx()  # 2つのプロットを関連付ける
    nnn=[i for i in range(len(rai[ad_st:ad_en,0]))]
    ax2.bar(nnn,rai[ad_st:ad_en,0],align="center",label="ave")
    ax2.set_yticks([0,20,40,60,80])
    ax2.set_ylim([80,0])
    plt.xlim([0,len(label_pred2017[ad_st:ad_en,0])])
    plt.title("peak±24h"+"  "+"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred2017[ad_st:ad_en,0],label_pred2017[ad_st:ad_en,1]))))    
    #ax2.legend()
    pdf.savefig()
    
    
    fig, ax1 = plt.subplots()
    maxi = label_pred2017[:,0].max()
    maxw = np.where(label_pred2017[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred2017[:,1].max()
    maxw_p = np.where(label_pred2017[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    ad_st = maxw-24
    ad_en = maxw+24+1
    ax1.plot(label_pred2017[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    ax1.plot(label_pred2017[ad_st:ad_en,1],label="pred",c="red",marker=".",linestyle="None")
    ax1.set_yticks([0,200,400,600,800,1000])
    ax1.set_ylim([-70,label_pred2017[:,0].max()+100])
    ax2 = ax1.twinx()  # 2つのプロットを関連付ける
    nnn=[i for i in range(len(rai[ad_st:ad_en,0]))]
    ax2.bar(nnn,rai[ad_st:ad_en,0],align="center",label="ave")
    ax2.set_yticks([0,20,40,60,80])
    ax2.set_ylim([80,0])
    plt.xlim([0,len(label_pred2017[ad_st:ad_en,0])])
    plt.title("peak±24h"+"  "+"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred2017[ad_st:ad_en,0],label_pred2017[ad_st:ad_en,1]))))    
    #ax2.legend()
    pdf.savefig()
    
    ###
    
    
    plt.figure()    
    maxi = 565
    maxw = np.where(label_pred2017[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred2017[5200:5300,1].max()
    maxw_p = np.where(label_pred2017[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    print(maxi_p)
    print(maxw_p)
    ad_st = maxw-50
    ad_en = maxw+50+1
    plt.figure()    
    plt.plot(label_pred2017[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    plt.plot(label_pred2017[ad_st:ad_en,1],label="pred",c="red",marker=".",linestyle="None")
    plt.xlim([0,len(label_pred2017[ad_st:ad_en,0])])
    plt.ylim([-70,label_pred2017[:,0].max()+100])
    plt.title("peak±50h"+"  "+"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred2017[ad_st:ad_en,0],label_pred2017[ad_st:ad_en,1]))))    
    plt.legend()
    pdf.savefig()
    
    
    plt.figure()    
    maxi = 753
    maxw = np.where(label_pred2017[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred2017[5200:5300,1].max()
    maxw_p = np.where(label_pred2017[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    print(maxi_p)
    print(maxw_p)
    ad_st = maxw-50
    ad_en = maxw+50+1
    plt.figure()    
    plt.plot(label_pred2017[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    plt.plot(label_pred2017[ad_st:ad_en,1],label="pred",c="red",marker=".",linestyle="None")
    plt.xlim([0,len(label_pred2017[ad_st:ad_en,0])])
    plt.ylim([-70,label_pred2017[:,0].max()+100])
    plt.title("peak±50h"+"  "+"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred2017[ad_st:ad_en,0],label_pred2017[ad_st:ad_en,1]))))    
    plt.legend()
    pdf.savefig()
    
    
    plt.figure()    
    maxi = label_pred2017[:,0].max()
    maxw = np.where(label_pred2017[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred2017[:,1].max()
    maxw_p = np.where(label_pred2017[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    ad_st = maxw-50
    ad_en = maxw+50+1
    plt.figure()    
    plt.plot(label_pred2017[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    plt.plot(label_pred2017[ad_st:ad_en,1],label="pred",c="red",marker=".",linestyle="None")
    plt.xlim([0,len(label_pred2017[ad_st:ad_en,0])])
    plt.title("peak±50h")    
    plt.ylim([-70,label_pred2017[:,0].max()+100])
    plt.text(5,900,"value_diff=" + "{0:.3f}".format(maxi-maxi_p))
    plt.text(5,800,"time_diff=" + str(maxw-maxw_p))   
    plt.text(5,1000,"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred2017[ad_st:ad_en,0],label_pred2017[ad_st:ad_en,1]))))
    plt.legend()
    pdf.savefig()
    
    ###
    #2軸#
    
    fig, ax1 = plt.subplots()
    maxi = 565
    maxw = np.where(label_pred2017[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred2017[5200:5300,1].max()
    maxw_p = np.where(label_pred2017[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    print(maxi_p)
    print(maxw_p)
    ad_st = maxw-50
    ad_en = maxw+50+1
    ax1.plot(label_pred2017[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    ax1.plot(label_pred2017[ad_st:ad_en,1],label="pred",c="red",marker=".",linestyle="None")
    ax1.set_yticks([0,200,400,600,800,1000])
    ax1.set_ylim([-70,label_pred2017[:,0].max()+100])
    ax2 = ax1.twinx()  # 2つのプロットを関連付ける
    nnn=[i for i in range(len(rai[ad_st:ad_en,0]))]
    ax2.bar(nnn,rai[ad_st:ad_en,0],align="center",label="ave")
    ax2.set_yticks([0,20,40,60,80])
    ax2.set_ylim([80,0])
    plt.xlim([0,len(label_pred2017[ad_st:ad_en,0])])
    plt.title("peak±50h"+"  "+"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred2017[ad_st:ad_en,0],label_pred2017[ad_st:ad_en,1]))))    
    #ax2.legend()
    pdf.savefig()
    
    
    fig, ax1 = plt.subplots()
    maxi = 753
    maxw = np.where(label_pred2017[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred2017[5200:5300,1].max()
    maxw_p = np.where(label_pred2017[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    print(maxi_p)
    print(maxw_p)
    ad_st = maxw-50
    ad_en = maxw+50+1
    ax1.plot(label_pred2017[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    ax1.plot(label_pred2017[ad_st:ad_en,1],label="pred",c="red",marker=".",linestyle="None")
    ax1.set_yticks([0,200,400,600,800,1000])
    ax1.set_ylim([-70,label_pred2017[:,0].max()+100])
    ax2 = ax1.twinx()  # 2つのプロットを関連付ける
    nnn=[i for i in range(len(rai[ad_st:ad_en,0]))]
    ax2.bar(nnn,rai[ad_st:ad_en,0],align="center",label="ave")
    ax2.set_yticks([0,20,40,60,80])
    ax2.set_ylim([80,0])
    plt.xlim([0,len(label_pred2017[ad_st:ad_en,0])])
    plt.title("peak±50h"+"  "+"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred2017[ad_st:ad_en,0],label_pred2017[ad_st:ad_en,1]))))    
    #ax2.legend()
    pdf.savefig()
    
    
    fig, ax1 = plt.subplots()
    maxi = label_pred2017[:,0].max()
    maxw = np.where(label_pred2017[:,0]==maxi)
    maxw = int(maxw[0])
    maxi_p = label_pred2017[:,1].max()
    maxw_p = np.where(label_pred2017[:,1]==maxi_p)
    maxw_p = int(maxw_p[0])
    ad_st = maxw-50
    ad_en = maxw+50+1
    ax1.plot(label_pred2017[ad_st:ad_en,0],label="label",c="black",linewidth = 1.5)
    ax1.plot(label_pred2017[ad_st:ad_en,1],label="pred",c="red",marker=".",linestyle="None")
    ax1.set_yticks([0,200,400,600,800,1000])
    ax1.set_ylim([-70,label_pred2017[:,0].max()+100])
    ax2 = ax1.twinx()  # 2つのプロットを関連付ける
    nnn=[i for i in range(len(rai[ad_st:ad_en,0]))]
    ax2.bar(nnn,rai[ad_st:ad_en,0],align="center",label="ave")
    ax2.set_yticks([0,20,40,60,80])
    ax2.set_ylim([80,0])
    plt.xlim([0,len(label_pred2017[ad_st:ad_en,0])])
    plt.title("peak±50h"+"  "+"RMSE(cm)=" + "{0:.3f}".format(np.sqrt(mean_squared_error(label_pred2017[ad_st:ad_en,0],label_pred2017[ad_st:ad_en,1]))))    
    #ax2.legend()
    pdf.savefig()

    pdf.close()
    
    return

In [ ]:
print("-"*100)
print("Done" + "!"*10)


# 重み50個のプロットを保存する

save_shimanto_results(st,en,path_val,path2017,label_pred_val,label_pred2017)